# Coauthors sequence in Astrophysics

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
data_path = os.getenv('DATA_PATH')
result_path = os.getenv('RESULT_PATH')
root = os.getenv('ROOT')

os.chdir(root)
os.getcwd()

'/disks/qnap3/users/23-miura/coauth-interest'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import random

from scipy.stats import expon
from scipy.stats import pareto

In [5]:
authors_valid_As = pd.read_pickle(result_path+"authors_valid_As.pickle")
num_coauth = authors_valid_As.groupby("eid")["authid"].nunique()
num_coauth.head()

eid
117    2
140    7
172    3
181    2
207    9
Name: authid, dtype: int64

In [ ]:
year_df = pd.read_pickle(data_path + "paper_detail/year.pickle")
print(year_df.head())

In [8]:
num_coauth = pd.merge(num_coauth,year_df,on="eid",how="left")
num_coauth.head()

eid
229021    1997
266363    1998
229022    1982
229023    1989
266364    1998
Name: year, dtype: int64


,authid,year
eid,,
117,2,1997
140,7,1993
172,3,1997
181,2,1998
207,9,1995


In [11]:
year_df.loc[[117,140,172,181,207]]

eid
117    1997
140    1993
172    1997
181    1998
207    1995
dtype: int64

In [12]:
num_coauth = num_coauth.sort_values(by="year")
m_seq = num_coauth["authid"].values
m_seq[:3]

array([4, 2, 4])

In [30]:
m_seq = m_seq[m_seq>1]

In [16]:
# import random 
# m_seq_sample = random.sample(m_seq,)

1838995

In [31]:
p=0.76
q=0.82
m=3

C_HIS = {0:{1,2},1:{0,2},2:{0,1}}
INC = {0,1,2}
coauth = []

def step(m):
  max_id = len(INC)
  id = set()
  inc_his = [] # team of previously selected agent
  
  while len(id)<m:
    # incumbents
    if random.random()<p:
      if random.random()<q and len(inc_his)!=0:
        choice =random.choice(list(inc_his))
      else:
        choice =random.choice(list(INC))
      id.add(choice)
      inc_his = C_HIS[choice]
    # newcomer
    else:
      id.add(max_id)# ID of the newcomer
      max_id += 1
  
  for i in id:
    if C_HIS.get(i)!=None:
      C_HIS[i].update(id - {i})
    else:
      C_HIS.update([(i,id - {i})])
  INC.update(id)
  return id

for i in range(10000):
  coauth.append(step(m_seq[i]))


In [32]:
coauth

[{1, 2, 3, 4},
 {0, 1},
 {0, 1, 3, 4},
 {1, 2},
 {1, 2, 3, 5},
 {2, 3, 6, 7},
 {5, 8},
 {5, 8, 9, 10},
 {0, 7, 11, 12},
 {1, 3},
 {3, 13},
 {5, 9},
 {4, 14},
 {4, 14},
 {1, 2, 4, 14},
 {13, 15},
 {0, 11, 13, 15, 16, 17},
 {0, 2, 3, 6, 7, 11, 13, 18, 19, 20, 21, 22, 23},
 {0, 4, 16},
 {7, 11},
 {15, 24},
 {8, 9},
 {5, 9, 10, 25},
 {3, 5, 19},
 {3, 13, 15},
 {2, 7, 13, 26},
 {3, 22},
 {1, 3, 7, 12, 15, 21},
 {18, 19},
 {11, 19, 20, 22, 23, 27, 28},
 {7, 23, 29},
 {1, 14, 30, 31},
 {5, 7, 9, 19, 23, 26, 27, 32},
 {15, 21, 32},
 {2, 4},
 {33, 34},
 {18, 28},
 {1, 35, 36},
 {15, 17},
 {13, 19},
 {12, 21},
 {0, 31, 33, 37},
 {0, 11, 16},
 {1, 3, 14, 15, 24, 30, 31, 33, 38, 39, 40},
 {24, 30, 33, 40, 41},
 {5, 40},
 {6, 13, 42, 43},
 {39, 44, 45, 46},
 {30, 39},
 {1, 6, 7, 35, 47, 48, 49, 50},
 {0, 12, 40},
 {12, 15, 51},
 {1, 36},
 {41, 52},
 {2, 9, 19, 20, 27},
 {33, 37},
 {0, 12},
 {7, 49, 53},
 {44, 45},
 {31, 33},
 {3, 24, 40, 54},
 {1, 24, 47, 55, 56},
 {53, 57},
 {22, 27},
 {1, 50},
 {

In [33]:
temp = pd.DataFrame(coauth).rename_axis("eid")
num_works = 200

def combine_columns(row):
    return row.values.tolist()

# author_df = temp.iterrows.values.tolist()
author_df = pd.DataFrame(temp.apply(combine_columns, axis=1).rename("authid"))
author_df = author_df.assign(year=[i // num_works for i in range(author_df.shape[0])])
author_df

,authid,year
eid,,
0,"[1.0, 2.0, 3.0, 4.0, nan, nan, nan, nan, nan, ...",0
1,"[0.0, 1.0, nan, nan, nan, nan, nan, nan, nan, ...",0
2,"[0.0, 1.0, 3.0, 4.0, nan, nan, nan, nan, nan, ...",0
3,"[1.0, 2.0, nan, nan, nan, nan, nan, nan, nan, ...",0
4,"[1.0, 2.0, 3.0, 5.0, nan, nan, nan, nan, nan, ...",0
...,...,...
9995,"[8184.0, 8185.0, nan, nan, nan, nan, nan, nan,...",49
9996,"[8186.0, 725.0, nan, nan, nan, nan, nan, nan, ...",49
9997,"[404.0, 2951.0, nan, nan, nan, nan, nan, nan, ...",49


In [34]:
from source.package.util import plotter

authors_valid = author_df.explode("authid").reset_index()
authors_valid
# coauthorship_count = authors_valid[['eid', 'authid']].groupby(by='authid').count()
# big_author = coauthorship_count.rename({"eid":"n_papers"},axis=1)
# big_author
# # plotter(big_author["n_papers"], count="number")

,eid,authid,year
0,0,1.0,0
1,0,2.0,0
2,0,3.0,0
3,0,4.0,0
4,0,NaN,0
...,...,...,...
289995,9999,NaN,49
289996,9999,NaN,49
289997,9999,NaN,49
289998,9999,NaN,49


In [35]:
def coauthor_works(id):
    works = authors_valid["eid"][authors_valid['authid']==id]
    
    coauthors = authors_valid[authors_valid['eid'].isin(works)].query(f"authid!={id}")
    df_seqyear_multi = coauthors.groupby(by='authid', group_keys=True).filter(lambda x: len(x) > 1)
    if len(df_seqyear_multi)<1: return None
    df_seqyear_delta = df_seqyear_multi.groupby(by='authid', group_keys=True)['year'] \
    .apply(lambda x: x - x.shift(1)) \
    .dropna().reset_index().rename(columns={"level_1":"eid"})
    
    df_seqyear_delta["source_authid"] = id
    df_seqyear_delta["previous_work"] = df_seqyear_delta.groupby("authid").shift(1)["eid"]
    
    df_seqyear_delta = df_seqyear_delta.dropna()
    return df_seqyear_delta


In [36]:
res = pd.DataFrame()
auth_list = [i for i in range(authors_valid["authid"].max())]
auth_list = random.sample(auth_list,len(auth_list))
for i in range(1000):
  res = pd.concat([res,coauthor_works(auth_list[i])],axis="index")
res

TypeError: 'float' object cannot be interpreted as an integer

In [37]:
plotter(res["eid"].nunique(),count="number")
print()

KeyError: 'eid'

In [ ]:
plotter(authors_valid.groupby("authid")["eid"].nunique(),count="number")
print()